In [16]:
# This is a sample Python script.

# Press Shift+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.
import nltk
from nltk import word_tokenize
import simplejson as jsoncm
import sklearn
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn import metrics

import numpy as np
import matplotlib.pyplot as plt


def create_bow_from_reviews(filename, min_pos=7, max_neg=4):
    print('\nLoading the file: \n', filename)
    with open(filename, 'r') as jfile:
        data = jsoncm.load(jfile)
    print('\nTotal number of reviews extracted =', len(data))

    text = []
    Y = []
    lengths = []
    print('\nExtracting tokens from each review.....(can be slow for a large number of reviews)......')
    for d in data:  # can substitute data[0:9] here if you want to test this function on just a few reviews
        review = d['review']  # keep only the text and label
        stars = d['rating']
        # ....write some simple logic to generate a binary score for each review
        if stars >= min_pos:
            score = 1
            text.append(review)
            Y.append(score)
        elif stars <= max_neg:
            score = 0
            text.append(review)
            Y.append(score)

    # create an instance of a CountVectorizer, using
    # (1) the standard 'english' stopword set
    # (2) only keeping terms in the vocabulary that occur in at least 1% of documents
    # (3) allowing both unigrams and bigrams in the vocabulary (use "ngram_range=(1,2)" to do this)
    vectorizer = CountVectorizer(stop_words='english', min_df=0.01, ngram_range=(1, 2))

    # create a sparse BOW array from 'text' using vectorizer
    X = vectorizer.fit_transform(text)

    # an alternative above would be to use TfIDF rather than counts - which is very simple to do (but not needed here)

    print('Data shape: ', X.shape)

    # you can uncomment this next line if you want to see the full list of tokens in the vocabulary
    # print('Vocabulary: ', vectorizer.get_feature_names())

    return X, Y, vectorizer



def DecisionTree_classification(X, Y, test_fraction):
    # should add comments defining what the inputs are what the function does

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_fraction, random_state=42)
    #  set the state of the random number generator so that we get the same results across runs when testing our code

    print('Number of training examples: ', X_train.shape[0])
    print('Number of testing examples: ', X_test.shape[0])
    print('Vocabulary size: ', X_train.shape[1])

    # Specify the logistic classifier model with an l2 penalty for regularization and with fit_intercept turned on
    classifier = DecisionTreeClassifier(criterion="entropy",random_state=0)

    # Train a logistic regression classifier and evaluate accuracy on the training data
    print('\nTraining a model with', X_train.shape[0], 'examples.....')
    classifier.fit(X_train, Y_train)
    train_predictions = classifier.predict(X_train)
    train_accuracy = classifier.score(X_train, Y_train)
    print('\nTraining:')
    print(' accuracy:', format(100 * train_accuracy, '.2f'))

    # Compute and print accuracy and AUC on the test data
    print('\nTesting: ')
    test_predictions = classifier.predict(X_test)
    test_accuracy = classifier.score(X_test, Y_test)
    print(' accuracy:', format(100 * test_accuracy, '.2f'))

    class_probabilities = classifier.predict_proba(X_test)
    test_auc_score = metrics.roc_auc_score(Y_test, class_probabilities[:, 1])
    print(' AUC value:', format(100 * test_auc_score, '.2f'))

    return (classifier)


X, Y , vectorizer_BOW = create_bow_from_reviews('reviews.json')
# run a decision Tree classifier on the reviews, specifying the fraction to be used for testing
test_fraction = 0.5
DecisionTree_classifier = DecisionTree_classification(X, Y,test_fraction)





Loading the file: 
 reviews.json

Total number of reviews extracted = 1000

Extracting tokens from each review.....(can be slow for a large number of reviews)......
Data shape:  (1000, 1819)
Number of training examples:  500
Number of testing examples:  500
Vocabulary size:  1819

Training a model with 500 examples.....

Training:
 accuracy: 100.00

Testing: 
 accuracy: 67.40
 AUC value: 61.10
